In [1]:
%load_ext autoreload 

import pandas as pd
import numpy as np
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torch.utils.data import Dataset, DataLoader

from collections import defaultdict
from IPython.display import clear_output
from IPython.core.debugger import set_trace
import matplotlib.pyplot as plt
from itertools import combinations
from tqdm import tqdm

from sklearn.manifold import LocallyLinearEmbedding, Isomap, SpectralEmbedding
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.multioutput import RegressorChain, MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error
from sklearn.linear_model import Ridge, Lasso
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

from utils import load_dataset, calculate_position_variables

import warnings
warnings.filterwarnings("ignore")
%autoreload 2

- В кольцевом треке (circle_track) и треке с отверстиями (holy_track).    
 - В каждом из треков n= 4 мыши (CA1_22, CA1_23, CA1_24 и CA1_25) провели 3 раза по 15 минут с интервалом в 1 сутки (отсюда индексы 1D, 2D, 3D).   
 - В треке с отверстиями их число менялось от 3х в первый день до 1го в третий.  
 
Данные имеют вид .csv таблиц (разделители — запятые), в первом столбце- время в секундах, далее идут столбцы с координатами центра масс мыши (x,y), затем координаты цветных маркеров на голове мыши (x_green, y_green, x_red, y_red; расположены как на рисунке в приложениии);   
Они есть только для кольцевого трека и фиксируются камерой не очень надёжно, так что в некоторых местах имеют разрывы — я старался их очистить, насколько это возможно, но не всё удалось отфильтровать, к сожалению.   

Координаты даны в пикселях, диаметр трека равен 50 см. После координат маркеров следуют столбцы с собственно нейронной активностью (т.е. интенсивностью флуоресценции;  данные получены путём применения алгоритма CNMF-E) для каждой из клеток.  
 
Для кольцевого трека есть аналогичные «сырым» данным нейронной активности данные по дискретным активациям каждой из клеток  — они состоят из амплитуд значимых (>4 MAD) событий в местах, где они происходят и нулей во всех остальных точках (по формату всё то же, что и с «сырыми» данными, только без координат). 

![sdf](./mouse_setup.png)

In [2]:
TRIMS = {
    1:{'22':[2000,10], # done 
       '23':[60,200],  # done
       '24':[2500,1600],# done
       '25':[50,800]}, # done
    2:{'22':[100,20], # done
       '23':[10,1], # done
       '24':[50,900], # done
       '25':[500,800]}, # done
    3:{'22':[100,100], # done
       '23':[50,800], # done 
       '24':[50,1000], # done 
       '25':[50,600]} # done
}

MOUSES = ['22', '23', '24', '25']
DAYS = [1,2,3]
root = 'data'
track = 'Circle' 

# Estimate trim boundaries 

In [3]:
# fig1, axes1 = plt.subplots(ncols=len(DAYS), nrows=len(MOUSES), figsize=(20,20), dpi=300)
# fig2, axes2 = plt.subplots(ncols=len(DAYS), nrows=len(MOUSES), figsize=(20,20), dpi=300)

# for i, mouse in enumerate(MOUSES):
#     for j, day in enumerate(DAYS):
        
#         calcium_df = pd.read_csv(os.path.join(root,f"{track}/data/CA1_{mouse}_{day}D_initial_data.csv"), 
#                              index_col=0)
#         T = calcium_df.shape[0]
#         trim0, trim1 = TRIMS[day][mouse]

#         coords = calcium_df[['x','y']].values # [T,2]
#         coords -= coords.mean(0)[None,:]
#         coords_ = MinMaxScaler((-1,1)).fit_transform(coords)
#         phi = np.arctan2(coords_[:,1], coords_[:,0])
#         phi[phi < 0] = 2*np.pi + phi[phi < 0]
#         phi_ = MinMaxScaler((0,1)).fit_transform(phi[:,None]).flatten()
        
#         axes1[i,j].scatter(coords_[trim0:-trim1,0], coords_[trim0:-trim1,1], c=phi_[trim0:-trim1])
#         axes1[i,j].set_title(f'M_{mouse}_D_{day}')
    
#         coord_norms = np.linalg.norm(coords_, axis=1)
        
#         axes2[i,j].plot(coord_norms) 
#         axes2[i,j].vlines([trim0, T-trim1], min(coord_norms), max(coord_norms), color='red')        
#         axes2[i,j].set_title(f'M_{mouse}_D_{day}')
    
# plt.show()

In [4]:
# track = 'Circle'
# mouse = '24'
# day = 3

# TRIMS[day][mouse]

In [5]:
# calcium_df = pd.read_csv(os.path.join(root,f"{track}/data/CA1_{mouse}_{day}D_initial_data.csv"), 
#                      index_col=0)

# coords = calcium_df[['x','y']].values # [T,2]
# coords_, phi_, speed_, acceleration_ = calculate_position_variables(coords)
# coord_norms = np.linalg.norm(coords_, axis=1)

# offset_0 = 50
# offset_1 = 1000 #600

# plt.figure(figsize=(15,5),dpi=200)
# plt.plot(coord_norms[offset_0:-offset_1])
# plt.show()


# plt.figure(figsize=(15,5),dpi=200)
# plt.plot(speed_[offset_0:-offset_1])
# plt.show()


# plt.figure(figsize=(10,10),dpi=200)
# plt.scatter(coords_[offset_0:-offset_1,0], coords_[offset_0:-offset_1,1], c=phi_[offset_0:-offset_1])
# plt.show()
    

# Cadata PCA

In [24]:
variance_thresh = 0.99
pca_dims = defaultdict(dict)

fig, axes = plt.subplots(ncols=len(DAYS), nrows=len(MOUSES), figsize=(30,30), dpi=300)

for i, mouse in enumerate(MOUSES):
    for j, day in enumerate(DAYS):
        
        data, targets = load_dataset(root, mouse=mouse, day=day, track=track, trims=TRIMS[day][mouse])        
        
        X = data['cadata']
        X -= X.mean(1, keepdims=True)
        pca = PCA()
        pca.fit(X)
        d = X.shape[1]
        ev = pca.explained_variance_ratio_
        ev_cs = np.cumsum(ev)
        dim_pca = (np.arange(d)[ev_cs > variance_thresh])[0]
        
        pca_dims[mouse][day] = dim_pca
        
        axes[i,j].plot(ev_cs)
        axes[i,j].set_title(f'PCA dim:{dim_pca}, \n M_{mouse}_D_{day}')
        
plt.show()

In [25]:
np.save(f'./results/pca_dims_cadata_{variance_thresh}', pca_dims)